In [2]:
from google.colab import files

uploaded = files.upload()



Saving data_cleaning  (1).xlsx to data_cleaning  (1).xlsx


In [3]:
import pandas as pd
import numpy as np
import re


In [4]:
file_name = list(uploaded.keys())[0]
xls = pd.ExcelFile(file_name)
usage_raw = pd.read_excel(xls, sheet_name="Usage")
contracts_raw = pd.read_excel(xls, sheet_name="Contracts")


In [6]:
header_text = usage_raw.columns[0]
rows = usage_raw.iloc[:, 0].astype(str)

def parse_row(text):
    text = text.strip()
    kwh = float(re.search(r'([0-9]*\.[0-9]+)\s*kwh', text, re.I).group(1))
    hour = re.search(r'(\d{1,2}\s*AM|\d{1,2}\s*PM|\d{1,2}AM|\d{1,2}PM)', text, re.I).group(1).replace(" ", "").upper()
    date = re.search(r'(\d{1,2}(?:st|nd|rd|th)?-[A-Za-z]{3}-\d{4})', text).group(1)
    return hour, date, kwh

def clean_date(d):
    return pd.to_datetime(re.sub(r'(st|nd|rd|th)', '', d), format="%d-%b-%Y")

h_hour, h_date, h_kwh = parse_row(header_text)


In [7]:
records = [{"hour_label": h_hour, "date": clean_date(h_date), "kwh": h_kwh}]
for r in rows:
    hour, d, kwh = parse_row(r)
    records.append({"hour_label": hour, "date": clean_date(d), "kwh": kwh})

usage = pd.DataFrame(records)


In [8]:
def hour_from_label(label):
    n = int(label[:-2])
    if label[-2:] == "AM":
        return 0 if n == 12 else n
    return 12 if n == 12 else n + 12

usage["hour"] = usage["hour_label"].apply(hour_from_label)
usage["datetime"] = usage["date"] + pd.to_timedelta(usage["hour"], unit="h")
usage = usage.sort_values("datetime").reset_index(drop=True)

usage["month"] = usage["datetime"].dt.month
usage["month_name"] = usage["datetime"].dt.month_name()
usage["weekday"] = usage["datetime"].dt.day_name()


In [9]:
contracts = contracts_raw.dropna(how="all")

monthly = contracts.loc[contracts["Unnamed: 4"].notna(), ["Unnamed: 4", "Unnamed: 5"]]
monthly = monthly[monthly["Unnamed: 4"] != "Monthly Flex"]
monthly_rates = dict(zip(monthly["Unnamed: 4"], monthly["Unnamed: 5"]))

flat_rate = float(contracts.loc[contracts["Unnamed: 0"] == "Flat Rate", "Unnamed: 1"].iloc[0])

hourly = contracts.loc[contracts["Unnamed: 8"].notna(), ["Unnamed: 8", "Unnamed: 9"]]
hourly = hourly[hourly["Unnamed: 8"] != "Hourly Flex"]

hourly["hour"] = hourly["Unnamed: 8"].apply(lambda x: hour_from_label(x.split("until")[0].strip()))
hourly_rates = dict(zip(hourly["hour"], hourly["Unnamed: 9"]))

usage["rate_monthly"] = usage["month_name"].map(monthly_rates)
usage["rate_hourly"] = usage["hour"].map(hourly_rates)


In [10]:
q1 = usage["kwh"].mean()
q2 = usage.loc[usage["month"] == 2, "kwh"].mean()
q3 = usage.groupby("weekday")["kwh"].mean().idxmax()
q4 = usage["kwh"].rolling(4).sum().max()
q5 = (usage["kwh"] * usage["rate_monthly"]).sum()
q6 = {
    "No Flex": usage["kwh"].sum() * flat_rate,
    "Monthly Flex": q5,
    "Hourly Flex": (usage["kwh"] * usage["rate_hourly"]).sum()
}
q1, q2, q3, q4, q5, min(q6, key=q6.get)


(np.float64(0.7817481735159817),
 np.float64(0.8337347242921013),
 'Sunday',
 17.237000000000002,
 np.float64(1421.2057699999998),
 'Hourly Flex')

In [26]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


In [27]:
from google.colab import files
uploaded = files.upload()


Saving train (1).csv to train (1) (1).csv
Saving test (1).csv to test (1) (1).csv


In [29]:
train_file = None
test_file = None

for k in uploaded.keys():
    name = k.lower()
    if "train" in name:
        train_file = k
    if "test" in name:
        test_file = k

train = pd.read_csv(train_file)
test = pd.read_csv(test_file)

X = train.drop(columns=["y"])
y = train["y"]

num_cols = X.select_dtypes(include=[np.number]).columns
cat_cols = X.select_dtypes(include=["object"]).columns

num_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc", StandardScaler())
])

cat_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("oh", OneHotEncoder(handle_unknown="ignore"))
])

prep = ColumnTransformer([
    ("num", num_pipe, num_cols),
    ("cat", cat_pipe, cat_cols)
])

model = RandomForestRegressor(
    n_estimators=400,
    random_state=42,
    n_jobs=-1
)

pipe = Pipeline([
    ("prep", prep),
    ("model", model)
])

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

pipe.fit(X_train, y_train)
valid_pred = pipe.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, valid_pred))
rmse


np.float64(0.020073740155610618)

In [30]:
pipe.fit(X, y)
test_pred = pipe.predict(test)

idx_vals = test["existence expectancy index"].values
potential = -np.log(idx_vals + 0.01) + 3
score = potential ** 2

n = len(test)
energy = np.zeros(n)
total = 50000.0
max_per = 100.0

low_mask = idx_vals < 0.7
low_idx = np.where(low_mask)[0]
low_sorted = low_idx[np.argsort(-score[low_mask])]

need_low = 0.1 * total

for i in low_sorted:
    if need_low <= 0:
        break
    give = min(max_per, need_low)
    energy[i] += give
    need_low -= give

remain = total - energy.sum()
all_sorted = np.argsort(-score)

for i in all_sorted:
    if remain <= 0:
        break
    cap = max_per - energy[i]
    if cap <= 0:
        continue
    give = min(cap, remain)
    energy[i] += give
    remain -= give

submission = pd.DataFrame({
    "index": np.arange(len(test)),
    "pred": test_pred,
    "opt_pred": energy
})

submission.to_csv("submission.csv", index=False)


In [31]:
files.download("submission.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>